In [1]:
from exact_synthesis.prec import synthesize_z_rotation
from exact_synthesis.prec import evaluate_gate_sequence, rz_mp
from exact_synthesis.prec import d
from exact_synthesis.synthesis import transpile_ft_sigma
import mpmath
from mpmath import mp

/home/jkr/projects/topological-quantum-compiling/src/exact_synthesis/numberTheory.py:366: SyntaxWarning: invalid escape sequence '\i'
  """


In [2]:
input = lambda k : 2 * mpmath.pi * k / (10**3)
error = lambda k : 10**(-k)

C,U = synthesize_z_rotation(input(1), error(4))
print(len(C))
print(C)

Equations is not going to be solvable
Equations is not going to be solvable
Equations is not going to be solvable
Equations is not going to be solvable
Equations is not going to be solvable
Equations is not going to be solvable
Equations is not going to be solvable
FOUND SOLUTION
1
56
[WIGate(power=1), TGate(power=6), FGate(), TGate(power=5), FGate(), TGate(power=9), FGate(), TGate(power=9), FGate(), TGate(power=9), FGate(), TGate(power=9), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=4), FGate(), TGate(power=1), FGate(), TGate(power=1), FGate(), TGate(power=2), FGate(), TGate(power=4), FGate(), TGate(power=1), FGate(), TGate(power=8), FGate(), TGate(power=9), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=2), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=4), FGate(), TGate(power=8), FGate(), TGate(power=2), FGate(), TGate(power=4), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=3)]


In [3]:
dist = d(input(1),U.u, U.v)

dist < error(4)
dist - error(4)

mpf('0.0098919997541270805')

In [4]:
mpmath.dps = 200
C,U = synthesize_z_rotation(input(10), 1e-5)
print(len(C))
dist=d(input(4),U.u, U.v)
dist

Equations is not going to be solvable
FOUND SOLUTION
1
58


mpf('0.0094060448186145569')

In [5]:
print(C)

[WIGate(power=9), TGate(power=6), FGate(), TGate(power=8), FGate(), TGate(power=8), FGate(), TGate(power=1), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=1), FGate(), TGate(power=8), FGate(), TGate(power=9), FGate(), TGate(power=2), FGate(), TGate(power=1), FGate(), TGate(power=8), FGate(), TGate(power=2), FGate(), TGate(power=9), FGate(), TGate(power=5), FGate(), TGate(power=8), FGate(), TGate(power=5), FGate(), TGate(power=1), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=1), FGate(), TGate(power=9), FGate(), TGate(power=1), FGate(), TGate(power=2), FGate(), TGate(power=1), FGate(), TGate(power=6), FGate(), TGate(power=8), FGate(), TGate(power=6), FGate(), TGate(power=7)]


In [7]:
CC =transpile_ft_sigma(C)
len(CC)

660

In [ ]:
from mpmath import matrix